# PyTorch Implementation of the NRMS Model for `EBNeRD` (RecSys'24 Challenge)
### Course: `02456 Deep Learning` (Fall 2024)  
**Institution:** Technical University of Denmark (DTU)  
**Authors:** Kevin Moore (s204462) and Nico Tananow (s[insert number])

### Acknowledgments  
1. Special thanks to **Johannes Kruse** for his [TensorFlow implementation of the NRMS Model](https://github.com/ebanalyse/ebnerd-benchmark), which greatly supported the development of this PyTorch implementation for the EBNeRD project.  


2. Our implementation is based on the NRMS model described in the paper **["Neural News Recommendation with Multi-Head Self-Attention"](https://aclanthology.org/D19-1671/)** by Wu et al. (2019).

# Implementation

## 1. Importing Dependencies
Import all necessary libraries and modules, including `utils.model` and `utils.helper` for the NRMS model, data preparation, training, and evaluation.

In [1]:
import sys
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # to avoid warnings in transformers
from pathlib import Path
# Get the current directory
current_dir = os.getcwd()
root_dir = os.path.join(os.path.dirname(os.path.dirname(current_dir)), "dtu-02456-deep-learning-ebnerd")
src_dir = os.path.join(root_dir, "src")
# Append the relative path to the utils folder and ebrec src
sys.path.append(os.path.join(current_dir, "utils"))
sys.path.append(src_dir)
from importlib import reload

import torch
import utils.model
reload(utils.model)
from utils.model import (
    NRMSModel
)

import utils.helper
reload(utils.helper)
from utils.helper import (
    HParams,
    load_articles_and_embeddings,
    prepare_training_data,
    prepare_test_data,
    train_model,
    evaluate_model,
)


## 2. Setting Hyperparameters
Initialize the hyperparameters for the model

In [2]:
# Setting hyperparameters
hparams = HParams()
hparams.data_fraction = 0.01
hparams.batch_size = 32

DATASPLIT = "ebnerd_small"

### 3. Loading Data
1. Loading and creating **embeddings** for articles
2. Loading **training** data and splitting it to training/validation (from `{datasplit}/train`)
3. Loading **testing** data for final evaluation (from `{datasplit}/validation`)

In [3]:
PATH = Path(os.path.join(current_dir, "data"))
print("Loading data from ", PATH, "with datasplit:", DATASPLIT)

# Loading articles and embeddings
article_mapping, word_embeddings = load_articles_and_embeddings(hparams, PATH)

# Training Data
train_loader, val_loader = prepare_training_data(
    hparams, PATH, DATASPLIT, article_mapping
)

# Test Data for final evaluation
test_loader = prepare_test_data(
    hparams, PATH, DATASPLIT, article_mapping
)

Loading data from  /Users/kevinmoore/Git Repositories/dtu-02456-deep-learning-ebnerd/our_implementation/data with datasplit: ebnerd_large


KeyboardInterrupt: 

### 4. Training the Model
- Initialize the NRMS model with preloaded hyperparameters and embeddings. 
- Train the model using the training and validation datasets. 
- Early stopping is applied with a patience parameter of 3 to prevent overfitting.

In [4]:
print("Training model with ", hparams)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NRMSModel(hparams, word_embeddings)

model = train_model(
    device,
    model,
    train_loader,
    val_loader,
    hparams,
    patience=3,
)

Training model with  
 title_size: 30
 head_num: 20
 head_dim: 20
 attention_hidden_dim: 200
 dropout: 0.2
 batch_size: 32
 verbose: False
 data_fraction: 0.01
 sampling_nratio: 4
 history_size: 20
 epochs: 1
 learning_rate: 0.001
 transformer_model_name: facebookai/xlm-roberta-base
Training information saved to: checkpoints/2024-12-13T11-32-57/info.txt


Validation: 100%|██████████| 10/10 [00:01<00:00,  6.38batch/s]


Epoch 1/1, Train Loss: 1.6011, Val Loss: 1.6047, Val AUC: 0.5302, Improvement from Previous Epoch: 0.5302
Checkpoint saved to: checkpoints/2024-12-13T11-32-57/nrms_checkpoint_1.pth


### 5. Evaluating the Model
- Evaluate the trained NRMS model on the testing dataset. 
- Print out performance metrics

In [5]:
# Evaluate model
metrics = evaluate_model(model, test_loader, device)
print("\nValidation Metrics:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

Testing: 100%|██████████| 77/77 [00:28<00:00,  2.71batch/s]


Validation Metrics:
auc: 0.5448
